In [10]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import glob
%matplotlib inline

In [24]:
sub = 'sub-09'
ses = 'ses-01'
run = 'run-01'
battery = 'camcan'
# task_source = 'gng'
task = 'stop-nogo'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
datadir = os.path.join(main_dir, sub, battery, ses)
os.chdir(datadir)
csv_files = glob.glob(task + '*.txt')
tsv_files = []
for i in range(len(csv_files)):
    tsv_files.append(task + '_' + sub + '_run-0' + str(i+1) + '.tsv')

In [22]:
csv_files

['stop-nogo_sub-09_run-02_30920-1041.txt',
 'stop-nogo_sub-09_run-01_30920-1030.txt']

In [25]:
tsv_files

['stop-nogo_sub-09_run-01.tsv', 'stop-nogo_sub-09_run-02.tsv']

In [41]:
# Read in the rest of the file
file_df = pd.read_csv(os.path.join(datadir, csv_files[1]), sep='\t', skipinitialspace=True)

In [42]:
file_df

,tr#,type,arrLR,Stype,Sdel,butt,resp,resp2,RT,RT2,RTsum,eval,relStart
0,1,0,1,0,0,1,1,0,523,0,0,1,0
1,2,0,2,0,0,2,2,0,397,0,0,1,2523
2,3,1,2,3,300,2,0,2,0,218,518,2,5045
3,4,0,1,0,0,1,1,0,339,0,0,1,7591
4,5,1,2,2,250,2,0,0,0,0,0,1,10114
...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,236,0,1,0,0,1,1,0,547,0,0,1,594246
236,237,0,2,0,0,2,2,0,476,0,0,1,596769
237,238,0,2,0,0,2,2,0,437,0,0,1,599292
238,239,1,2,3,400,2,0,2,0,56,456,2,601814


In [29]:
file_df.keys()

Index(['tr#', 'type', 'arrLR', 'Stype', 'Sdel', 'butt', 'resp', 'resp2', 'RT',
       'RT2', 'RTsum', 'eval', 'relStart'],
      dtype='object')

In [43]:
# Extract the onsets of stimulus, probe and response
trial_onsets = (file_df['relStart'] + 500)

# Find the 4 trials types:
# 1. Go trials ('type' = 0)
# 2. No-go trials ('type' = 1 and 'Stype' = 1)
# 3. Successful stop trials ('type' = 1 and 'Stype' = 2 or 3 and RT1 = 0 and RT2 = 0)
# 3. Unsuccessful stop trials ('type' = 1 and 'Stype' = 2 or 3 and (RT1 != 0 or RT2 != 0))
go_trials = file_df['type'] == 0
unsucgo_trials = file_df.loc[go_trials][file_df['RT'] == 0].index
nogo_trials = (file_df['type'] == 1) & (file_df['Stype'] == 1)
unsucnogo_trials = file_df.loc[nogo_trials][file_df['RT'] != 0].index
sucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] == 0)
unsucstop_trials = (file_df['type'] == 1) & ((file_df['Stype'] == 2) | (file_df['Stype'] == 3)) & \
            (file_df['RT2'] != 0)

# Calculate trial durations
durations = np.array(np.ones(len(trial_onsets)), dtype=np.int_)
durations[go_trials] = file_df.loc[go_trials]['RT']
durations[unsucgo_trials] = 2500
durations[nogo_trials] = 2500
durations[unsucnogo_trials] = file_df.loc[unsucnogo_trials]['RT']
durations[sucstop_trials] = 2500
durations[unsucstop_trials] = file_df.loc[unsucstop_trials]['RTsum']

# Assign trial types
trial_types = np.array(np.ones(len(trial_onsets)), dtype=np.object_)
trial_types[go_trials] = 'go'
trial_types[nogo_trials] = 'nogo'
trial_types[sucstop_trials] = 'successful_stop'
trial_types[unsucstop_trials] = 'unsuccessful_stop'

/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:10: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  # Remove the CWD from sys.path while we load stuff.
/home/sshankar/env/fastsrm/lib/python3.6/site-packages/ipykernel_launcher.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if sys.path[0] == '':


In [37]:
trial_types

array(['go', 'unsuccessful_stop', 'go', 'go', 'go', 'go', 'go', 'go',
       'go', 'go', 'go', 'successful_stop', 'go', 'go', 'successful_stop',
       'go', 'go', 'go', 'go', 'go', 'nogo', 'go', 'go', 'go', 'go',
       'unsuccessful_stop', 'nogo', 'go', 'go', 'go', 'go', 'go', 'go',
       'go', 'go', 'unsuccessful_stop', 'go', 'go', 'go', 'go', 'go',
       'go', 'go', 'go', 'nogo', 'successful_stop', 'go', 'go', 'go',
       'unsuccessful_stop', 'go', 'go', 'nogo', 'nogo', 'nogo', 'go',
       'go', 'successful_stop', 'go', 'successful_stop', 'go',
       'unsuccessful_stop', 'successful_stop', 'successful_stop', 'go',
       'successful_stop', 'go', 'go', 'nogo', 'unsuccessful_stop', 'go',
       'go', 'go', 'go', 'go', 'unsuccessful_stop', 'go', 'go', 'go',
       'go', 'go', 'go', 'go', 'go', 'unsuccessful_stop', 'go', 'nogo',
       'go', 'go', 'go', 'successful_stop', 'unsuccessful_stop', 'go',
       'successful_stop', 'go', 'go', 'go', 'go', 'unsuccessful_stop',
       'nogo

In [38]:
trial_onsets

0         500
1        3023
2        5568
3        8091
4       10614
        ...  
235    594702
236    597225
237    599748
238    602271
239    604793
Name: relStart, Length: 240, dtype: int64

In [39]:
durations

array([ 526,  402,  512,  441,  423,  490,  472,  470,  557,  445,  516,
       2500,  537,  516, 2500,  501,  440,  514,  553,  494, 2500,  509,
        508,  425,  397,  430, 2500,  409,  506,  599,  500,  639,  481,
        573,  492,  464,  419,  441,  618,  601,  386,  520,  639,  561,
       2500, 2500,  595,  418,  509,  454,  508,  536, 2500, 2500, 2500,
        494,  500, 2500,  575, 2500,  425,  513, 2500, 2500,  505, 2500,
        537,  567, 2500,  680,  523,  710,  677,  582,  429,  437,  522,
        445,  498,  630,  478,  569,  507,  549,  451,  546, 2500,  505,
        408,  453, 2500,  573,  517, 2500,  508,  495,  559,  471,  597,
       2500,  628,  542,  540,  544,  636,  581,  560,  547,  494,  526,
        638,  429,  409, 2500,  569, 2500,  434,  520,  419,  387,  429,
        454,  515,  459,  467,  374,  531,  524,  409,  430,  549,  427,
        371,  459,  513, 2500, 2500,  572,  513,  503, 2500, 2500, 2500,
        465,  511,  491,  467,  508,  416,  412,  3

In [44]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':trial_onsets/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_files[1]), sep='\t', float_format='%0.3f', index=False)